In [1]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Thêm đường dẫn tới thư mục src
sys.path.append(os.path.abspath('../src'))

# Import hàm load dữ liệu tự viết
from data_loader import load_raw_data

print("Libraries imported & Path setup done.")

Libraries imported & Path setup done.


In [2]:
# Hàm này tự động tìm file trong data/raw/housing.csv
df = load_raw_data()

print(f"Kích thước ban đầu: {df.shape}")
df.head()

Loading data from: /home/bush/Personal/Github/mlp-california-housing-price-prediction/data/raw/housing.csv
Kích thước ban đầu: (20640, 10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
# 1. Xử lý Missing Values 
df_clean = df.dropna()
print(f"Đã xóa {len(df) - len(df_clean)} dòng bị thiếu dữ liệu.")

# 2. Xử lý cột 'ocean_proximity' bằng One-Hot Encoding
if 'ocean_proximity' in df_clean.columns:
    print("Đang thực hiện One-Hot Encoding cho cột 'ocean_proximity'...")
    
    # pd.get_dummies sẽ tự động tách cột chữ thành nhiều cột số (0 và 1)
    # dtype=float để đảm bảo dữ liệu là số thực (0.0, 1.0) cho nhân ma trận
    df_clean = pd.get_dummies(df_clean, columns=['ocean_proximity'], dtype=float)

print(f"Kích thước sau khi xử lý: {df_clean.shape}")
print("Các cột hiện tại:", df_clean.columns.tolist())

# Xem thử 5 dòng đầu để thấy sự thay đổi
display(df_clean.head())

Đã xóa 207 dòng bị thiếu dữ liệu.
Đang thực hiện One-Hot Encoding cho cột 'ocean_proximity'...
Kích thước sau khi xử lý: (20433, 14)
Các cột hiện tại: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity_<1H OCEAN', 'ocean_proximity_INLAND', 'ocean_proximity_ISLAND', 'ocean_proximity_NEAR BAY', 'ocean_proximity_NEAR OCEAN']


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0.0,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0.0,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0.0,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0.0,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0.0,0.0,0.0,1.0,0.0


In [4]:
# Target column
target_col = 'median_house_value'

# Tách X (Features): Lấy tất cả trừ cột target
X = df_clean.drop(target_col, axis=1).values

# Tách y (Target): Lấy cột target và reshape thành cột dọc (N, 1)
y_raw = df_clean[target_col].values.reshape(-1, 1)

# === QUAN TRỌNG: SCALE TARGET ===
# Giá nhà gốc rất lớn (ví dụ 450,000). Nếu để nguyên, Loss sẽ rất to -> Tràn bộ nhớ.
# Ta chia cho 100,000 để đưa về khoảng [0, 5].
SCALE_FACTOR = 100000.0
y = y_raw / SCALE_FACTOR

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Ví dụ giá nhà gốc: {y_raw[0][0]}")
print(f"Ví dụ giá nhà sau khi scale: {y[0][0]}")

X shape: (20433, 13)
y shape: (20433, 1)
Ví dụ giá nhà gốc: 452600.0
Ví dụ giá nhà sau khi scale: 4.526


In [5]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42 # Cố định random seed để kết quả giống nhau mỗi lần chạy
)

print(f"Số lượng mẫu huấn luyện (Train): {X_train_raw.shape[0]}")
print(f"Số lượng mẫu kiểm thử (Test): {X_test_raw.shape[0]}")

Số lượng mẫu huấn luyện (Train): 16346
Số lượng mẫu kiểm thử (Test): 4087


In [6]:
scaler = StandardScaler()

# 1. Học từ tập Train và biến đổi tập Train
X_train = scaler.fit_transform(X_train_raw)

# 2. Dùng thông số đã học để biến đổi tập Test
X_test = scaler.transform(X_test_raw)

print("Đã chuẩn hóa dữ liệu xong!")
print(f"Mean của X_train (xấp xỉ 0): {np.mean(X_train):.2f}")
print(f"Std của X_train (xấp xỉ 1): {np.std(X_train):.2f}")

Đã chuẩn hóa dữ liệu xong!
Mean của X_train (xấp xỉ 0): -0.00
Std của X_train (xấp xỉ 1): 1.00


In [7]:
# Tạo thư mục nếu chưa có
processed_dir = os.path.join('..', 'data', 'processed')
os.makedirs(processed_dir, exist_ok=True)

# Đường dẫn lưu file
path_X_train = os.path.join(processed_dir, 'X_train.npy')
path_y_train = os.path.join(processed_dir, 'y_train.npy')
path_X_test = os.path.join(processed_dir, 'X_test.npy')
path_y_test = os.path.join(processed_dir, 'y_test.npy')

# Lưu file
np.save(path_X_train, X_train)
np.save(path_y_train, y_train)
np.save(path_X_test, X_test)
np.save(path_y_test, y_test)

print(f"Đã lưu dữ liệu vào thư mục: {os.path.abspath(processed_dir)}")

Đã lưu dữ liệu vào thư mục: /home/bush/Personal/Github/mlp-california-housing-price-prediction/data/processed


In [8]:
# Kiểm tra số lượng nhà theo từng loại địa hình
print(df['ocean_proximity'].value_counts())

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64


In [9]:
# Thử load lại 1 file để check
check_X = np.load(path_X_train)

if check_X.shape == X_train.shape:
    print("Kiểm tra thành công: File lưu và file gốc khớp nhau.")
else:
    print("Cảnh báo: Có lỗi khi lưu file!")

Kiểm tra thành công: File lưu và file gốc khớp nhau.
